WSGI：Web Server Gateway Interface。

WSGI接口定义非常简单，它只要求Web开发者实现一个函数，就可以响应HTTP请求。我们来看一个最简单的Web版本的“Hello, web!”：

In [ ]:
def application(environ, start_response):
    start_response('200 OK', [('Content-Type', 'text/html')])
    return '<h1>Hello, web!</h1>'

django中对应的实现在`django/core/handlers/wsgi.py `中，

In [ ]:
class WSGIHandler(base.BaseHandler):                                                                                                              
    request_class = WSGIRequest                                                                                                                   
                                                                                                                                                  
    def __init__(self, *args, **kwargs):                                                                                                          
        super().__init__(*args, **kwargs)                                                                                                         
        self.load_middleware()                                                                                                                    
                                                                                                                                                  
    def __call__(self, environ, start_response):                                                                                                  
        set_script_prefix(get_script_name(environ))                                                                                               
        signals.request_started.send(sender=self.__class__, environ=environ)                                                                      
        request = self.request_class(environ)                                                                                                     
        response = self.get_response(request)                                                                                                     
                                                                                                                                                  
        response._handler_class = self.__class__                                                                                                  
                                                                                                                                                  
        status = '%d %s' % (response.status_code, response.reason_phrase)                                                                         
        response_headers = list(response.items())                                                                                                 
        for c in response.cookies.values():                                                                                                       
            response_headers.append(('Set-Cookie', c.output(header='')))                                                                          
        start_response(status, response_headers)                                                                                                  
        if getattr(response, 'file_to_stream', None) is not None and environ.get('wsgi.file_wrapper'):                                            
            response = environ['wsgi.file_wrapper'](response.file_to_stream)                                                                      
        return response 